# Mars StyleGAN

The [Basic Gan](https://github.com/kheyer/MarsGAN/blob/master/Basic%20Gan.ipynb) notebook showed that a very simple generator/discriminator setup can generate fake Mars landscapes that are poor quality but still recognizable. The basic GAN suffered from several flaws. The images generated were blurry, pixilated and full of artifacts. The images were small, 64x64 pixels. Training a model of the same architecture to create larger images proved unstable. In this notebook, we use a more sophistocated generator to create higher quality images at larger sizes. We are going to use the famous [StyleGAN](https://arxiv.org/pdf/1812.04948.pdf).

StyleGAN is noteworthy because it was the first GAN model to disentangle the latent factors used to generate an image. Most GAN models start with a random vector that is used to generate an image. StyleGAN does things differently. A random latent vector is first sent through a mapping network consisting of several fully connected layers. This allows the generator to learn an intermediate latent space that is best for generating images. The intermediate latent vector is then injected into the generator at different layers.

![](media/stylegan.png)

The key finding of StyleGAN was that adding the intermediate latent vector at different layers of the generator caused different effects on the output image. Injection at early layers of the generator influenced major details of the image, while injection at later layers influenced finer details. The result is that the StyleGAN generator can be used to blend different latent vectors.

![](media/stylegan_mixing.jpg)

StyleGAN is able to train with good stability to images as large as 1024x1024 by training with [progressive growing](https://arxiv.org/pdf/1710.10196.pdf). We start training on 4x4 images. After some time, we grow to 8x8, then 16x16 and so on. When we grow the model, there's a small issue. We now have a new layer that converts a tensor of activations to a 3 channel RGB image. Since this new layer is untrained, it likely produces poor quality images. To prevent this from messing up training, the new RGB layer is phased in.

![](media/rgb.png)

So that's a quick overview. Lets start looking at the model.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.vision.gan import *
from fastai.callbacks import *

In [3]:
path = Path('G:/Mars/')

In [4]:
def get_data(bs, size, path, num_workers, noise_size=512):
    return (GANItemList.from_folder(path, noise_sz=noise_size)
               .split_none()
               .label_from_func(noop)
               .transform(tfms=[[crop_pad(size=size, row_pct=(0,1), col_pct=(0,1))], []], size=size, tfm_y=True)
               .databunch(bs=bs, num_workers=num_workers)
               .normalize(stats = [torch.tensor([0.5,0.5,0.5]), torch.tensor([0.5,0.5,0.5])], do_x=False, do_y=True))

In [5]:
# I resized the main dataset into individual sets for each image size
# This has a noticeable effect on training speed
data_4 = get_data(256, 4, path/'patches_4', 8)
data_8 = get_data(256, 8, path/'patches_8', 8)
data_16 = get_data(128, 16, path/'patches_16', 8)
data_32 = get_data(128, 32, path/'patches_32', 8)
data_64 = get_data(48, 64, path/'patches_64', 8)
data_128 = get_data(22, 128, path/'patches_128', 8)
data_256 = get_data(10, 256, path/'patches_256', 8)
data_512 = get_data(4, 512, path/'image_patches', 8)

## StyleGAN Fundamentals



In [ ]:
class EqualLR:
    def __init__(self, name):
        self.name = name

    def compute_weight(self, module):
        weight = getattr(module, self.name + '_orig')
        fan_in = weight.data.size(1) * weight.data[0][0].numel()

        return weight * math.sqrt(2 / fan_in)

    @staticmethod
    def apply(module, name):
        fn = EqualLR(name)

        weight = getattr(module, name)
        del module._parameters[name]
        module.register_parameter(name + '_orig', nn.Parameter(weight.data))
        module.register_forward_pre_hook(fn)

        return fn

    def __call__(self, module, input):
        weight = self.compute_weight(module)
        setattr(module, self.name, weight)


def equal_lr(module, name='weight'):
    EqualLR.apply(module, name)

    return module

In [ ]:
class EqualConv2d(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()

        conv = nn.Conv2d(*args, **kwargs)
        conv.weight.data.normal_()
        conv.bias.data.zero_()
        self.conv = equal_lr(conv)

    def forward(self, input):
        return self.conv(input)


class EqualLinear(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()

        linear = nn.Linear(in_dim, out_dim)
        linear.weight.data.normal_()
        linear.bias.data.zero_()

        self.linear = equal_lr(linear)

    def forward(self, input):
        return self.linear(input)

In [ ]:
class ConvBlock(nn.Module):
    def __init__(
        self,
        in_channel,
        out_channel,
        kernel_size,
        padding,
        kernel_size2=None,
        padding2=None,
        pixel_norm=True,
        spectral_norm=False,
    ):
        super().__init__()

        pad1 = padding
        pad2 = padding
        if padding2 is not None:
            pad2 = padding2

        kernel1 = kernel_size
        kernel2 = kernel_size
        if kernel_size2 is not None:
            kernel2 = kernel_size2

        self.conv = nn.Sequential(
            EqualConv2d(in_channel, out_channel, kernel1, padding=pad1),
            nn.LeakyReLU(0.2),
            EqualConv2d(out_channel, out_channel, kernel2, padding=pad2),
            nn.LeakyReLU(0.2),
        )

    def forward(self, input):
        out = self.conv(input)

        return out

In [ ]:
class PixelNorm(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input):
        return input / torch.sqrt(torch.mean(input ** 2, dim=1, keepdim=True) + 1e-8)

In [ ]:
class AdaptiveInstanceNorm(nn.Module):
    def __init__(self, in_channel, style_dim):
        super().__init__()

        self.norm = nn.InstanceNorm2d(in_channel)
        self.style = EqualLinear(style_dim, in_channel * 2)

        self.style.linear.bias.data[:in_channel] = 1
        self.style.linear.bias.data[in_channel:] = 0

    def forward(self, input, style):
        style = self.style(style).unsqueeze(2).unsqueeze(3)
        gamma, beta = style.chunk(2, 1)

        out = self.norm(input)
        out = gamma * out + beta

        return out

In [ ]:
class NoiseInjection(nn.Module):
    def __init__(self, channel):
        super().__init__()

        self.weight = nn.Parameter(torch.zeros(1, channel, 1, 1))

    def forward(self, image, noise):
        return image + self.weight * noise


class ConstantInput(nn.Module):
    def __init__(self, channel, size=4):
        super().__init__()

        self.input = nn.Parameter(torch.randn(1, channel, size, size))

    def forward(self, input):
        batch = input.shape[0]
        out = self.input.repeat(batch, 1, 1, 1)

        return out

In [12]:
class StyledConvBlock(nn.Module):
    def __init__(
        self,
        in_channel,
        out_channel,
        kernel_size=3,
        padding=1,
        style_dim=512,
        initial=False,
    ):
        super().__init__()

        if initial:
            self.conv1 = ConstantInput(in_channel)

        else:
            self.conv1 = EqualConv2d(
                in_channel, out_channel, kernel_size, padding=padding
            )

        self.noise1 = equal_lr(NoiseInjection(out_channel))
        self.adain1 = AdaptiveInstanceNorm(out_channel, style_dim)
        self.lrelu1 = nn.LeakyReLU(0.2)

        self.conv2 = EqualConv2d(out_channel, out_channel, kernel_size, padding=padding)
        self.noise2 = equal_lr(NoiseInjection(out_channel))
        self.adain2 = AdaptiveInstanceNorm(out_channel, style_dim)
        self.lrelu2 = nn.LeakyReLU(0.2)

    def forward(self, input, style, noise):
        out = self.conv1(input)
        out = self.noise1(out, noise)
        out = self.adain1(out, style)
        out = self.lrelu1(out)

        out = self.conv2(out)
        out = self.noise2(out, noise)
        out = self.adain2(out, style)
        out = self.lrelu2(out)

        return out

In [13]:
class Generator(nn.Module):
    def __init__(self, code_dim):
        super().__init__()

        self.progression = nn.ModuleList(
            [
                StyledConvBlock(512, 512, 3, 1, initial=True),
                StyledConvBlock(512, 512, 3, 1),
                StyledConvBlock(512, 512, 3, 1),
                StyledConvBlock(512, 512, 3, 1),
                StyledConvBlock(512, 256, 3, 1),
                StyledConvBlock(256, 128, 3, 1),
                StyledConvBlock(128, 64, 3, 1),
                StyledConvBlock(64, 32, 3, 1),
                StyledConvBlock(32, 16, 3, 1),
            ]
        )

        self.to_rgb = nn.ModuleList(
            [
                EqualConv2d(512, 3, 1),
                EqualConv2d(512, 3, 1),
                EqualConv2d(512, 3, 1),
                EqualConv2d(512, 3, 1),
                EqualConv2d(256, 3, 1),
                EqualConv2d(128, 3, 1),
                EqualConv2d(64, 3, 1),
                EqualConv2d(32, 3, 1),
                EqualConv2d(16, 3, 1),
            ]
        )
        
    def forward(self, style, noise, step=0, alpha=-1, mixing_range=(-1, -1)):
        out = noise[0]

        if len(style) < 2:
            inject_index = [len(self.progression) + 1]

        else:
            inject_index = random.sample(list(range(step)), len(style) - 1)

        crossover = 0

        for i, (conv, to_rgb) in enumerate(zip(self.progression, self.to_rgb)):
            if mixing_range == (-1, -1):
                if crossover < len(inject_index) and i > inject_index[crossover]:
                    crossover = min(crossover + 1, len(style))

                style_step = style[crossover]

            else:
                if mixing_range[0] <= i <= mixing_range[1]:
                    style_step = style[1]

                else:
                    style_step = style[0]

            if i > 0 and step > 0:
                upsample = F.interpolate(
                    out, scale_factor=2, mode='bilinear', align_corners=False
                )
                # upsample = self.blur(upsample)
                out = conv(upsample, style_step, noise[i])

            else:
                out = conv(out, style_step, noise[i])

            if i == step:
                out = to_rgb(out)

                if i > 0 and alpha > 0: #0 <= alpha < 1:
                    skip_rgb = self.to_rgb[i - 1](upsample)
                    out = alpha * skip_rgb + (1 - alpha) * out

                break

        return out

In [14]:
class StyledGenerator(nn.Module):
    def __init__(self, code_dim=512, n_mlp=8):
        super().__init__()

        self.generator = Generator(code_dim)

        layers = [PixelNorm()]
        for i in range(n_mlp):
            layers.append(EqualLinear(code_dim, code_dim))
            layers.append(nn.LeakyReLU(0.2))

        self.style = nn.Sequential(*layers)
        
        self.step = 0
        self.alpha = 0

    def forward(self, input, noise=None, mean_style=None, style_weight=0, mixing_range=(-1, -1), mixing=True):
        
        bs, ch, _, _ = input.shape
        input = input.view(bs, ch)
        
        if self.step < 1:
            mixing=False
        
        if mixing and random.random() < 0.9:
            shuffle = torch.randperm(input.size(0)).to(input.device)
            input = [input, input[shuffle]]
        else:
            input = [input]  
        
        styles = []

        for i in input:
            styles.append(self.style(i))

        batch = input[0].shape[0]

        if noise is None:
            noise = []

            for i in range(self.step + 1):
                size = 4 * 2 ** i
                noise.append(torch.randn(batch, 1, size, size, device=input[0].device))

        if mean_style is not None:
            styles_norm = []

            for style in styles:
                styles_norm.append(mean_style + style_weight * (style - mean_style))

            styles = styles_norm

        return self.generator(styles, noise, self.step, self.alpha, mixing_range=mixing_range)

    def mean_style(self, input):
        style = self.style(input).mean(0, keepdim=True)

        return style
        
    def grow_model(self, step=None, alpha=None):
        if step:
            self.step = step
        else:
            self.step += 1
            
        if alpha is not None:
            self.alpha = alpha
        else:
            self.alpha = 1

In [15]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        self.progression = nn.ModuleList(
            [
                ConvBlock(16, 32, 3, 1),
                ConvBlock(32, 64, 3, 1),
                ConvBlock(64, 128, 3, 1),
                ConvBlock(128, 256, 3, 1),
                ConvBlock(256, 512, 3, 1),
                ConvBlock(512, 512, 3, 1),
                ConvBlock(512, 512, 3, 1),
                ConvBlock(512, 512, 3, 1),
                ConvBlock(513, 512, 3, 1, 4, 0),
            ]
        )

        self.from_rgb = nn.ModuleList(
            [
                EqualConv2d(3, 16, 1),
                EqualConv2d(3, 32, 1),
                EqualConv2d(3, 64, 1),
                EqualConv2d(3, 128, 1),
                EqualConv2d(3, 256, 1),
                EqualConv2d(3, 512, 1),
                EqualConv2d(3, 512, 1),
                EqualConv2d(3, 512, 1),
                EqualConv2d(3, 512, 1),
            ]
        )

        # self.blur = Blur()

        self.n_layer = len(self.progression)

        self.linear = EqualLinear(512, 1)
        
        self.step = 0
        self.alpha = 0

    def forward(self, input, actual=False):
        if actual:
            for i in range(self.step, -1, -1):
                index = self.n_layer - i - 1

                if i == self.step:
                    out = self.from_rgb[index](input)

                if i == 0:
                    out_std = torch.sqrt(out.var(0, unbiased=False) + 1e-8)
                    mean_std = out_std.mean()
                    mean_std = mean_std.expand(out.size(0), 1, 4, 4)
                    out = torch.cat([out, mean_std], 1)

                out = self.progression[index](out)

                if i > 0:
                    # out = F.avg_pool2d(out, 2)
                    out = F.interpolate(
                        out, scale_factor=0.5, mode='bilinear', align_corners=False
                    )

                    if i == self.step and self.alpha > 0:  #0 <= alpha < 1:
                        # skip_rgb = F.avg_pool2d(input, 2)
                        skip_rgb = self.from_rgb[index + 1](input)
                        skip_rgb = F.interpolate(
                            skip_rgb, scale_factor=0.5, mode='bilinear', align_corners=False
                        )

                        out = self.alpha * skip_rgb + (1 - self.alpha) * out

            out = out.squeeze(2).squeeze(2)
            out = self.linear(out)

            return (out, input)
        else:
            return (None, input)
    
    def grow_model(self, step=None, alpha=None):
        if step:
            self.step = step
        else:
            self.step += 1
            
        if alpha is not None:
            self.alpha = alpha
        else:
            self.alpha = 1

In [ ]:
class StyleCriticLoss_WGANGP(nn.Module):
    
    def forward(self, real, fake, gradient):
        loss = fake.mean() - real.mean() + 0.001 * real.pow(2).mean() + gradient
        return loss        
        
class StyleGenLoss_WGANGP(nn.Module):
    
    def forward(self, fake_pred, *args):
        return -fake_pred.mean()

In [ ]:
class StyleCriticLoss_R1(nn.Module):
    
    def forward(self, fake, gradient_predict):
        real_predict = F.softplus(-gradient_predict[0]).mean()
        
        grad_real = torch.autograd.grad(outputs=gradient_predict[0].sum(), inputs=gradient_predict[1], create_graph=True)[0]
        #grad_penalty = grad_real.pow(2).sum()
        grad_penalty = grad_real.view(grad_real.size(0), -1).norm(2, dim=1).pow(2).mean()
        grad_penalty = 10 / 2 * grad_penalty
        
        fake_predict = F.softplus(fake).mean()
        
        loss = real_predict + fake_predict + grad_penalty
        
        return loss
    
class StyleGenLoss_R1(nn.Module):
    
    def forward(self, fake_pred, *args):
        return F.softplus(-fake_pred).mean()

In [ ]:
class GANLossGradient(GANModule):
    "Wrapper around `loss_funcC` (for the critic) and `loss_funcG` (for the generator)."
    def __init__(self, loss_funcG:Callable, loss_funcC:Callable, gan_model:GANModule, grad_scale=10):
        super().__init__()
        self.loss_funcG,self.loss_funcC,self.gan_model = loss_funcG,loss_funcC,gan_model
        self.grad_scale = grad_scale

    def generator(self, output, target):
        "Evaluate the `output` with the critic then uses `self.loss_funcG` to combine it with `target`."
        fake_pred = self.gan_model.critic(output, actual=True)[0]
        return self.loss_funcG(fake_pred, target, output)

    def critic(self, real_pred, input):
        "Create some `fake_pred` with the generator from `input` and compare them to `real_pred` in `self.loss_funcD`."
        fake = self.gan_model.generator(input.requires_grad_(False)).requires_grad_(True)
        fake_pred = self.gan_model.critic(fake, actual=True)
        
        real_pred[1].requires_grad = True
        gradient_predict = self.gan_model.critic(real_pred[1], actual=True)
        
        return self.loss_funcC(fake_pred[0], gradient_predict)        

In [ ]:
class AlphaDecay(LearnerCallback):
    
    def __init__(self, learn, end_batch):
        super().__init__(learn)
        self.end_batch = end_batch
        self.initial_alpha = self.learn.model.generator.alpha
        
    def on_batch_begin(self, iteration, **kwargs):
        if self.learn.model.generator.alpha > 0:
            self.learn.model.generator.alpha = self.initial_alpha - iteration / self.end_batch
            self.learn.model.critic.alpha = self.initial_alpha - iteration / self.end_batch

In [ ]:
class StyleGANLearner(Learner):
    "A `Learner` suitable for GANs."
    def __init__(self, data:DataBunch, generator:nn.Module, critic:nn.Module, gen_loss_func:LossFunction,
                 crit_loss_func:LossFunction, switcher:Callback=None, gen_first:bool=False, switch_eval:bool=True,
                 show_img:bool=True, clip:float=None, alpha_batch=10, **learn_kwargs):
        gan = GANModule(generator, critic)
        loss_func = GANLossGradient(gen_loss_func, crit_loss_func, gan)
        switcher = ifnone(switcher, partial(FixedGANSwitcher, n_crit=5, n_gen=1))
        super().__init__(data, gan, loss_func=loss_func, callback_fns=[switcher], **learn_kwargs)
        trainer = GANTrainer(self, clip=clip, switch_eval=switch_eval, show_img=show_img)
        self.gan_trainer = trainer
        self.callbacks.append(trainer)
        self.callback_fns.append(partial(AlphaDecay, end_batch=alpha_batch))

    @classmethod
    def from_learners(cls, learn_gen:Learner, learn_crit:Learner, switcher:Callback=None,
                      weights_gen:Tuple[float,float]=None, **learn_kwargs):
        "Create a GAN from `learn_gen` and `learn_crit`."
        losses = gan_loss_from_func(learn_gen.loss_func, learn_crit.loss_func, weights_gen=weights_gen)
        return cls(learn_gen.data, learn_gen.model, learn_crit.model, *losses, switcher=switcher, **learn_kwargs)

    @classmethod
    def stylegan_wgan(cls, data:DataBunch, generator:nn.Module, critic:nn.Module, 
             switcher:Callback=None, clip:float=0.01, **learn_kwargs):
        "Create a WGAN from `data`, `generator` and `critic`."
        return cls(data, generator, critic, StyleGenLoss_WGANGP(), 
                       StyleCriticLoss_WGANGP(), switcher=switcher, clip=clip, **learn_kwargs)
    
    @classmethod
    def stylegan_r1(cls, data:DataBunch, generator:nn.Module, critic:nn.Module, 
             switcher:Callback=None, clip:float=0.01, **learn_kwargs):
        "Create a WGAN from `data`, `generator` and `critic`."
        return cls(data, generator, critic, StyleGenLoss_R1(), 
                       StyleCriticLoss_R1(), switcher=switcher, clip=clip, **learn_kwargs)

    def grow_model(self, data, step=None, alpha=None, alpha_batch=None):
        self.model.generator.grow_model(step=step, alpha=alpha)
        self.model.critic.grow_model(step=step, alpha=alpha)
        self.data = data
        
        if alpha_batch is not None:
            self.callback_fns[-1] = partial(AlphaDecay, end_batch=alpha_batch)